<a href="https://colab.research.google.com/github/feiduobaby/good-first-issue/blob/main/Capstone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import polars as pl
import os
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
!cp -r /content/drive/MyDrive/114239_nfl_competition_files_published_analytics_final /content/

In [5]:
# Paths and constants
input_path = '/content/114239_nfl_competition_files_published_analytics_final/train'
def read_selected_data(input_path):
    # Define the directory containing your data files

    # List three specific Parquet files you want to read
    selected_files = [f"input_2023_w0{i}.csv" for i in range(1,19)]
    # Load and filter the data from only the selected Parquet files
    dfs = []
    for file_name in selected_files:
        file_path = f'{input_path}/{file_name}'
        lazy_df = pl.scan_csv(file_path)
        df = lazy_df.collect()
        dfs.append(df)

    # Concatenate all dataframes into a single dataframe
    full_df = pl.concat(dfs)

    return full_df